# <center>Model Evaluation</center>

In [6]:
import pandas as pd
import numpy as np

In [11]:
path = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/automobileEDA.csv'
df = pd.read_csv(path)
#lets only use numeric data
df = df._get_numeric_data()
df.head()

,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,diesel,gas
0,3,122,88.6,0.811148,0.890278,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0,11.190476,0,1
1,3,122,88.6,0.811148,0.890278,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0,11.190476,0,1
2,1,122,94.5,0.822681,0.909722,52.4,2823,152,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0,12.368421,0,1
3,2,164,99.8,0.848630,0.919444,54.3,2337,109,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0,9.791667,0,1
4,2,164,99.4,0.848630,0.922222,54.3,2824,136,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0,13.055556,0,1


## 1. Training and Testing

In [15]:
y = df['price']
x = df.drop('price',axis=1)

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)

print("Number of Test Samples : {} \nNumber of Train Samples : {} ".format(x_test.shape[0], x_train.shape[0]))

Number of Test Samples : 31 
Number of Train Samples : 170 


In [22]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train[['horsepower']], y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [121]:
lr.predict(x_test[['horsepower']])

array([12146.83498197,  7272.45270007,  9961.7670625 ,  8617.10988128,
       15340.39578735,  6263.95981416, 20214.77806925, 10634.0956531 ,
       15172.3136397 ,  7608.61699537, 11306.42424371,  8617.10988128,
       15340.39578735, 16180.80652561, 14499.98504909, 12146.83498197,
        5591.63122355, 13659.57431083,  7440.53484772, 10129.84921015,
       10297.9313578 ,  7608.61699537, 26433.81753236, 11810.67068667,
        5927.79551886, 25425.32464645, 15340.39578735,  7272.45270007,
       11306.42424371,  7440.53484772, 15004.23149205])

In [26]:
#Calculate R^2 on training set
r_sq = lr.score(x_train[['horsepower']], y_train)
print('R-sq for training set : ',r_sq)

R-sq for training set :  0.6449517437659684


In [28]:
#Calculate R^2 on test set
r_sq = lr.score(x_test[['horsepower']], y_test)
print('R-sq for testing set : ',r_sq)

R-sq for testing set :  0.707688374146705


## 2. Cross Validation

Sometimes we do not have sufficient testing data; as a result, you may want to perform Cross-validation. Let's go over several methods that you can use for Cross-validation.

In [41]:
from sklearn.model_selection import cross_val_score

In [42]:
Rcross = cross_val_score(lr, x[['horsepower']], y, cv=4)
Rcross

array([0.7746232 , 0.51716687, 0.74785353, 0.04839605])

We can calculate the average and standard deviation of our estimate:

In [43]:
print("The mean of Rcross is {} and standard deviation is {}".format(Rcross.mean(),Rcross.std()))

The mean of Rcross is 0.522009915042119 and standard deviation is 0.2911839444756029


In [44]:
from sklearn.model_selection import cross_val_predict
yhat = cross_val_predict(lr,x[['horsepower']], y, cv=4)
yhat[0:5]

array([14141.63807508, 14141.63807508, 20814.29423473, 12745.03562306,
       14762.35027598])

## 3. Ridge Regression

In [125]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

In [126]:
pr = PolynomialFeatures(degree=2)

In [127]:
x_train_pr = pr.fit_transform(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])
x_test_pr = pr.fit_transform(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])

In [128]:
RidgeModel = Ridge(alpha=0.1)

In [129]:
RidgeModel.fit(x_train_pr, y_train)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=6.43365e-17): result may not be accurate.
  overwrite_a=True).T


Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [134]:
yhat = RidgeModel.predict(x_test_pr)

In [136]:
#compare the predicted samples to our test set
print('Predicted:', yhat[0:4])
print('Test set :', y_test[0:4].values)

Predicted: [ 9589.40015921  6514.23158088 10604.73769887  5137.02179828]
Test set : [ 9549.  6229. 10245.  7295.]


We select the value of Alfa that minimizes the test error, for example, we can use a for loop.

In [137]:
Rsqu_test = []
Rsqu_train = []
dummy1 = []
ALFA = 10 * np.array(range(0,1000))
for alfa in ALFA:
    RigeModel = Ridge(alpha=alfa) 
    RigeModel.fit(x_train_pr, y_train)
    Rsqu_test.append(RigeModel.score(x_test_pr, y_test))
    Rsqu_train.append(RigeModel.score(x_train_pr, y_train))
    
print("R-sq for testing data {} \nR-Sq for training data {}".format(Rsqu_test[0:5],Rsqu_train[0:5]))

R-sq for testing data [0.824246840563103, 0.8234194740593661, 0.824009947134607, 0.8244788768783977, 0.8248440085892932] 
R-Sq for training data [0.8611992175540811, 0.8606371353453676, 0.8605944340754162, 0.8605531773500145, 0.8605081384602357]


## 4. Grid Search

In [138]:
from sklearn.model_selection import GridSearchCV

In [139]:
parameters1 = [{'alpha' : [0.001,0.1,1, 10, 100, 1000, 10000, 100000, 100000]}]

In [140]:
RR = Ridge()

In [141]:
Grid = GridSearchCV(RR, parameters1, cv=4)

In [142]:
Grid.fit(x[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']],y)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': [0.001, 0.1, 1, 10, 100, 1000, 10000, 100000, 100000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [143]:
BestRR = Grid.best_estimator_
BestRR

Ridge(alpha=10000, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)